## TEMP-PILOT EXPERIMENT OVERVIEW

    The rivers-pilot project runs the realistic SSC, v201812 (good bloom timing), from the following run directory:
        SS-run-sets/SalishSea/nemo3.6/tereza/RIVERS/TEPLO
    
    This is comparable to RIVERS_PILOT

    Things that need to change (add 2 degrees):
        -atmospheric forcing DONE
        -rivers forcing DONE
            
            temperature is in rivers_ConsTemp_month_plus2.nc which is in rivers-climatology under vc
        
        -boundary condition forcing 
            LiveOcean DONE
            north DONE - needs to go into tracers/north, called in namelist.lateral
        -initial condition (restart) forcing DONE
        
        
        
        
    Restarts are from /results/SalishSea/nowcast-green.201812/31dec16
    
    There are two run scenarios, base and SHIFT, which use identical namelist.times and have the following diff patterns per yamls (base v SHIFT):
                < run_id: RP_base_1
            ---
            > run_id: RP_base_1_SHIFT
            20c20
            <     link to: $PROJECT/tjarnik/MEOPAR/RIVERS_files/daily_R201702/
            ---
            >     link to: $PROJECT/tjarnik/MEOPAR/RIVERS_files/daily_45Dearly/
            54c54
            <   - ./namelist.atmos_rivers
            ---
            >   - ./namelist.atmos_riversSHIFT
            
    The RIVERS_files are made in: 
        notebooks/RIVER_PAPER/RIVERS_pilot/make_daily_files and make_daily_files_shifted_hydrograph
    These use daily interpolated values from monthly averages, for all rivers except the fraser, where realistic 2017 values are used. 
    
    Then we shift the 'glacial' watersheds, as well as the Fraser river, forward 45 days to examine effects. 
        (not the rest of the fraser watershed; this is because Tereza forgot. 
        Not really a problem though, for a pilot run)
    The glacial watersheds are: wat_to_rot = ['skagit', 'howe', 'bute', 'toba']
    
    Results are stored in: /data/tjarniko/results/RIV_PIL
    
## NOTEBOOK OVERVIEW

    1. Here we first visualize the rivers in the SalishSeaCast model by watershed. 
    
    2. We then look at hydrograph patterns by watershed, identifying 
    the largest 5 rivers in each watershed by runoff.
    
    3. We then introduce the river_acid_observatory tool, which shows the hydgrograph of each river of interest 
    as well as key inorg. C cycle variables in the vicinity of its mouth: 
    S, nitrate, diatoms, TA-DIC, DIC, nDIC, and Omega.
    We consider two radii of importance: 3 grid cells from river mouth (~1.5 km) and 10 grid cells (5 km)
    The exception is the Fraser, where we have to consider a point not at the river mouth. 
    
    4. We apply the river_acid_observatory tool to the largest river in each watershed, 
    for both the BASE and the SHIFT runs. 
    
    5. We also build a river_flux_observatory tool, which shows the hydrograph of each river of interest 
    as well as key carbon air-sea flux variables in the vicinity of its mouth:  
    pCO2, delta-pCO2, wind strength, and flux. We apply this to the largest rivers in the watershed as well. 
    

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import sys
import numpy as np
import netCDF4 as nc
import cmocean as cm
import matplotlib.pyplot as plt
import pickle
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)
sys.path.append('/data/tjarniko/mocsy')
import mocsy
import arrow
import gsw
import glob
sys.path.append('/data/tjarniko/MEOPAR/tools/SalishSeaTools/salishsea_tools/')

import river_201702 as rv

#### make restart file with tn+2

In [3]:
w = nc.Dataset('/results/SalishSea/nowcast-green.201812/31dec16/SalishSea_02473200_restart.nc')
# print(w)
# print(w['sn'][0,:,250,250])

temp = w['tn'][:]
newtemp = np.copy(temp)
newtemp[:] = newtemp[:]+2

# print(w['tn'][0,:,250,250])
# print(newtemp[0,:,250,250])
newtemp[newtemp==2] = 0

#print(newtemp[0,:,250,250])
#SalishSea_02473200_restart_trc.nc

In [4]:
# ### code to copy the new rivers file into its own .nc
# dsin = w
# newrivfile = f"./TEPLOFILES/SalishSea_TEPLO_02473200_restart.nc"
# ncname = newrivfile
# print(newrivfile)

# dsout = nc.Dataset(ncname, "w", format="NETCDF4")
# #Copy dimensions
# for dname, the_dim in dsin.dimensions.items():
#     #print(dname)
#     #print(len(the_dim))
#     dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
# # Copy variables
# for v_name, varin in dsin.variables.items():
#     #print(v_name)
#     outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
#     #print(varin.datatype)
#     outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
#     outVar[:] = varin[:]
# dsout['tn'][:] = newtemp
# dsout.close()

### make the rivers temperature file - need to only change one file, in rivers-climatology

In [5]:
# w = nc.Dataset('/data/tjarniko/MEOPAR/rivers-climatology/rivers_ConsTemp_month.nc')
# print(w['rotemper'][:,250,250])

# newtemp = np.copy(w['rotemper'][:])
# newtemp[:] = newtemp[:]+2
# print(w['rotemper'][:,250,250])
# print(newtemp[:,250,250])

# ## code to copy the new rivers file into its own .nc
# dsin = w
# newrivfile = f"./TEPLOFILES/rivers_ConsTemp_month.nc"
# ncname = newrivfile
# print(newrivfile)

# dsout = nc.Dataset(ncname, "w", format="NETCDF4")
# #Copy dimensions
# for dname, the_dim in dsin.dimensions.items():
#     #print(dname)
#     #print(len(the_dim))
#     dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
# # Copy variables
# for v_name, varin in dsin.variables.items():
#     #print(v_name)
#     outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
#     #print(varin.datatype)
#     outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
#     outVar[:] = varin[:]
# dsout['rotemper'][:] = newtemp
# dsout.close()

## make northern boundary climatology - needs to go in /tracers/north

In [13]:
w = nc.Dataset('/data/tjarniko/MEOPAR/tracers/north/Dosser_north_TEOS10.nc')
print(w)

newtemp = np.copy(w['votemper'][:])
newtemp[:] = newtemp[:]+2
print(np.shape(newtemp))
print(newtemp[0,:,0,0])
newtemp[newtemp==2] = 0
print(newtemp[0,:,0,0])

## code to copy the new rivers file into its own .nc
dsin = w
newrivfile = f"./TEPLOFILES/Dosser_north_TEOS10_plus2.nc"
ncname = newrivfile
print(newrivfile)

dsout = nc.Dataset(ncname, "w", format="NETCDF4")
#Copy dimensions
for dname, the_dim in dsin.dimensions.items():
    #print(dname)
    #print(len(the_dim))
    dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
# Copy variables
for v_name, varin in dsin.variables.items():
    #print(v_name)
    outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
    #print(varin.datatype)
    outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
    outVar[:] = varin[:]
dsout['votemper'][:] = newtemp
dsout.close()

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    title: Temperature & Salinity Boundary Conditions for Northern Boundary
    institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
    source: https://bitbucket.org/salishsea/tools/src/tip/I_ForcingFiles/OBC/DosserNorthernTS.ipynb
    history: [2018-07-14 14:42:33] Created netCDF4 zlib=True dataset.
    comment: T & S data from Hayley Dosser monthly climatology, TEOS10
    references: tracers/north/Dosser_north_TEOS10.nc
    dimensions(sizes): xbT(30), yb(10), deptht(40), time_counter(12)
    variables(dimensions): float32 deptht(deptht), float32 time_counter(time_counter), float32 votemper(time_counter,deptht,yb,xbT), float32 vosaline(time_counter,deptht,yb,xbT)
    groups: 

(12, 40, 10, 30)
[9.406303  9.40962   9.412938  9.416256  9.419573  9.422891  9.4262085
 9.429527  9.433107  9.436968  9.440834  9.444707  9.448597  9.451725
 9.45397

# LiveOcean

In [15]:
start ='2017-01-01'
end ='2017-12-31'
verbose = False
start_run = arrow.get(start)
end_run = arrow.get(end)

arrow_array = []

for r in arrow.Arrow.span_range('day', start_run, end_run):
    arrow_array.append(r)


for date in range(0,len(arrow_array)):
    print(date)
    #get appropriate date and open old river file
    tdate = arrow_array[date][0]
    y = tdate.format('YYYY')
    m = tdate.format('MM')
    d = tdate.format('DD')
    tday = f"y{y}m{m}d{d}"
    #print(tday)
    trivfile = f"/results/forcing/LiveOcean/boundary_conditions/LiveOcean_v201712_y{y}m{m}d{d}.nc"
    triv = nc.Dataset(trivfile)
    #print(triv)
    
    ## open rorunoff file and copy it
    rotemp = triv['votemper'][:]
    if verbose == True:
        print(np.shape(rotemp))
    new_rotemp = np.copy(rotemp)
    new_rotemp[:] = new_rotemp + 2
    
    new_rotemp[new_rotemp ==2] =0

    
    ### code to copy the new rivers file into its own .nc
    dsin = triv
    newrivfile = f"./TEPLOFILES/LiveOcean/LiveOcean_v201712_y{y}m{m}d{d}.nc"
    ncname = newrivfile
    print(newrivfile)
    
    dsout = nc.Dataset(ncname, "w", format="NETCDF4")
    #Copy dimensions
    for dname, the_dim in dsin.dimensions.items():
        #print(dname)
        #print(len(the_dim))
        dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
    # Copy variables
    for v_name, varin in dsin.variables.items():
        #print(v_name)
        outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
        #print(varin.datatype)
        outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
        outVar[:] = varin[:]
    dsout['votemper'][:] = new_rotemp
    dsout.close()

0
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d01.nc
1
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d02.nc
2
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d03.nc
3
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d04.nc
4
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d05.nc
5
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d06.nc
6
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d07.nc
7
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d08.nc
8
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d09.nc
9
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d10.nc
10
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d11.nc
11
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d12.nc
12
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d13.nc
13
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d14.nc
14
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d15.nc
15
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d16.nc
16
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m01d17.nc
17
./TE

142
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m05d23.nc
143
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m05d24.nc
144
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m05d25.nc
145
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m05d26.nc
146
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m05d27.nc
147
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m05d28.nc
148
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m05d29.nc
149
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m05d30.nc
150
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m05d31.nc
151
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m06d01.nc
152
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m06d02.nc
153
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m06d03.nc
154
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m06d04.nc
155
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m06d05.nc
156
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m06d06.nc
157
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m06d07.nc
158
./TEPLOFILES/LiveOcean/LiveOcean_v20

./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d07.nc
280
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d08.nc
281
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d09.nc
282
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d10.nc
283
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d11.nc
284
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d12.nc
285
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d13.nc
286
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d14.nc
287
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d15.nc
288
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d16.nc
289
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d17.nc
290
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d18.nc
291
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d19.nc
292
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d20.nc
293
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d21.nc
294
./TEPLOFILES/LiveOcean/LiveOcean_v201712_y2017m10d22.nc
295
./TEPLOFILES/LiveOcean/LiveOcean_v201712

## operational atmo

In [18]:
start ='2017-01-01'
end ='2017-12-31'
verbose = False
start_run = arrow.get(start)
end_run = arrow.get(end)

arrow_array = []

for r in arrow.Arrow.span_range('day', start_run, end_run):
    arrow_array.append(r)


for date in range(0,len(arrow_array)):
    print(date)
    #get appropriate date and open old river file
    tdate = arrow_array[date][0]
    y = tdate.format('YYYY')
    m = tdate.format('MM')
    d = tdate.format('DD')
    tday = f"y{y}m{m}d{d}"
    #print(tday)
    trivfile = f"/results/forcing/atmospheric/GEM2.5/operational/ops_y{y}m{m}d{d}.nc"
    triv = nc.Dataset(trivfile)
    print(triv)
    
    ## open rorunoff file and copy it
    rotemp = triv['tair'][:]
    if verbose == True:
        print(np.shape(rotemp))
    new_rotemp = np.copy(rotemp)
    new_rotemp[:] = new_rotemp + 2
    
    #new_rotemp[new_rotemp ==2] =0

    
    ### code to copy the new rivers file into its own .nc
    dsin = triv
    newrivfile = f"./TEPLOFILES/operational/ops_y{y}m{m}d{d}.nc"
    ncname = newrivfile
    print(newrivfile)
    
    dsout = nc.Dataset(ncname, "w", format="NETCDF4")
    #Copy dimensions
    for dname, the_dim in dsin.dimensions.items():
        #print(dname)
        #print(len(the_dim))
        dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
    # Copy variables
    for v_name, varin in dsin.variables.items():
        #print(v_name)
        outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
        #print(varin.datatype)
        outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
        outVar[:] = varin[:]
    dsout['tair'][:] = new_rotemp
    dsout.close()

0
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    History: Sun Jan  1 11:16:37 2017: ncks -4 -L4 -O /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m01d01.nc /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m01d01.nc
created by wgrib2
    GRIB2_grid_template: 20
    NCO: 4.4.2
    dimensions(sizes): time_counter(24), y(266), x(256)
    variables(dimensions): float32 atmpres(time_counter,y,x), float64 nav_lat(y,x), float64 nav_lon(y,x), float32 precip(time_counter,y,x), float32 qair(time_counter,y,x), float32 solar(time_counter,y,x), float32 tair(time_counter,y,x), float32 therm_rad(time_counter,y,x), float64 time_counter(time_counter), float32 u_wind(time_counter,y,x), float32 v_wind(time_counter,y,x), float64 x(x), float64 y(y)
    groups: 

./TEPLOFILES/operational/ops_y2017m01d01.nc
1
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    Hist

9
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    History: Tue Jan 10 11:12:06 2017: ncks -4 -L4 -O /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m01d10.nc /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m01d10.nc
created by wgrib2
    GRIB2_grid_template: 20
    NCO: 4.4.2
    dimensions(sizes): time_counter(24), y(266), x(256)
    variables(dimensions): float32 atmpres(time_counter,y,x), float64 nav_lat(y,x), float64 nav_lon(y,x), float32 precip(time_counter,y,x), float32 qair(time_counter,y,x), float32 solar(time_counter,y,x), float32 tair(time_counter,y,x), float32 therm_rad(time_counter,y,x), float64 time_counter(time_counter), float32 u_wind(time_counter,y,x), float32 v_wind(time_counter,y,x), float64 x(x), float64 y(y)
    groups: 

./TEPLOFILES/operational/ops_y2017m01d10.nc
10
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    His

18
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    History: Thu Jan 19 11:10:01 2017: ncks -4 -L4 -O /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m01d19.nc /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m01d19.nc
created by wgrib2
    GRIB2_grid_template: 20
    NCO: 4.4.2
    dimensions(sizes): time_counter(24), y(266), x(256)
    variables(dimensions): float32 atmpres(time_counter,y,x), float64 nav_lat(y,x), float64 nav_lon(y,x), float32 precip(time_counter,y,x), float32 qair(time_counter,y,x), float32 solar(time_counter,y,x), float32 tair(time_counter,y,x), float32 therm_rad(time_counter,y,x), float64 time_counter(time_counter), float32 u_wind(time_counter,y,x), float32 v_wind(time_counter,y,x), float64 x(x), float64 y(y)
    groups: 

./TEPLOFILES/operational/ops_y2017m01d19.nc
19
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    Hi

27
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    History: Sat Jan 28 11:07:02 2017: ncks -4 -L4 -O /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m01d28.nc /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m01d28.nc
created by wgrib2
    GRIB2_grid_template: 20
    NCO: 4.4.2
    dimensions(sizes): time_counter(24), y(266), x(256)
    variables(dimensions): float32 atmpres(time_counter,y,x), float64 nav_lat(y,x), float64 nav_lon(y,x), float32 precip(time_counter,y,x), float32 qair(time_counter,y,x), float32 solar(time_counter,y,x), float32 tair(time_counter,y,x), float32 therm_rad(time_counter,y,x), float64 time_counter(time_counter), float32 u_wind(time_counter,y,x), float32 v_wind(time_counter,y,x), float64 x(x), float64 y(y)
    groups: 

./TEPLOFILES/operational/ops_y2017m01d28.nc
28
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    Hi

36
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    History: Mon Feb  6 15:20:00 2017: ncks -4 -L4 -O /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m02d06.nc /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m02d06.nc
created by wgrib2
    GRIB2_grid_template: 20
    NCO: 4.4.2
    dimensions(sizes): time_counter(24), y(266), x(256)
    variables(dimensions): float32 atmpres(time_counter,y,x), float64 nav_lat(y,x), float64 nav_lon(y,x), float32 precip(time_counter,y,x), float32 qair(time_counter,y,x), float32 solar(time_counter,y,x), float32 tair(time_counter,y,x), float32 therm_rad(time_counter,y,x), float64 time_counter(time_counter), float32 u_wind(time_counter,y,x), float32 v_wind(time_counter,y,x), float64 x(x), float64 y(y)
    groups: 

./TEPLOFILES/operational/ops_y2017m02d06.nc
37
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    Hi

...

325
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    History: Wed Nov 22 10:37:32 2017: ncks -4 -L4 -O /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m11d22.nc /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m11d22.nc
created by wgrib2
    GRIB2_grid_template: 20
    NCO: 4.4.2
    dimensions(sizes): time_counter(24), y(266), x(256)
    variables(dimensions): float32 atmpres(time_counter,y,x), float64 nav_lat(y,x), float64 nav_lon(y,x), float32 percentcloud(time_counter,y,x), float32 precip(time_counter,y,x), float32 qair(time_counter,y,x), float32 solar(time_counter,y,x), float32 tair(time_counter,y,x), float32 therm_rad(time_counter,y,x), float64 time_counter(time_counter), float32 u_wind(time_counter,y,x), float32 v_wind(time_counter,y,x), float64 x(x), float64 y(y)
    groups: 

./TEPLOFILES/operational/ops_y2017m11d22.nc
326
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file fo

333
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    History: Thu Nov 30 10:37:14 2017: ncks -4 -L4 -O /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m11d30.nc /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m11d30.nc
created by wgrib2
    GRIB2_grid_template: 20
    NCO: 4.4.2
    dimensions(sizes): time_counter(24), y(266), x(256)
    variables(dimensions): float32 atmpres(time_counter,y,x), float64 nav_lat(y,x), float64 nav_lon(y,x), float32 percentcloud(time_counter,y,x), float32 precip(time_counter,y,x), float32 qair(time_counter,y,x), float32 solar(time_counter,y,x), float32 tair(time_counter,y,x), float32 therm_rad(time_counter,y,x), float64 time_counter(time_counter), float32 u_wind(time_counter,y,x), float32 v_wind(time_counter,y,x), float64 x(x), float64 y(y)
    groups: 

./TEPLOFILES/operational/ops_y2017m11d30.nc
334
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file fo

341
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    History: Fri Dec  8 10:37:19 2017: ncks -4 -L4 -O /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m12d08.nc /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m12d08.nc
created by wgrib2
    GRIB2_grid_template: 20
    NCO: 4.4.2
    dimensions(sizes): time_counter(24), y(266), x(256)
    variables(dimensions): float32 atmpres(time_counter,y,x), float64 nav_lat(y,x), float64 nav_lon(y,x), float32 percentcloud(time_counter,y,x), float32 precip(time_counter,y,x), float32 qair(time_counter,y,x), float32 solar(time_counter,y,x), float32 tair(time_counter,y,x), float32 therm_rad(time_counter,y,x), float64 time_counter(time_counter), float32 u_wind(time_counter,y,x), float32 v_wind(time_counter,y,x), float64 x(x), float64 y(y)
    groups: 

./TEPLOFILES/operational/ops_y2017m12d08.nc
342
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file fo

349
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    History: Sat Dec 16 10:37:25 2017: ncks -4 -L4 -O /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m12d16.nc /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m12d16.nc
created by wgrib2
    GRIB2_grid_template: 20
    NCO: 4.4.2
    dimensions(sizes): time_counter(24), y(266), x(256)
    variables(dimensions): float32 atmpres(time_counter,y,x), float64 nav_lat(y,x), float64 nav_lon(y,x), float32 percentcloud(time_counter,y,x), float32 precip(time_counter,y,x), float32 qair(time_counter,y,x), float32 solar(time_counter,y,x), float32 tair(time_counter,y,x), float32 therm_rad(time_counter,y,x), float64 time_counter(time_counter), float32 u_wind(time_counter,y,x), float32 v_wind(time_counter,y,x), float64 x(x), float64 y(y)
    groups: 

./TEPLOFILES/operational/ops_y2017m12d16.nc
350
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file fo

357
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.0
    History: Sun Dec 24 10:36:48 2017: ncks -4 -L4 -O /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m12d24.nc /results/forcing/atmospheric/GEM2.5/operational/ops_y2017m12d24.nc
created by wgrib2
    GRIB2_grid_template: 20
    NCO: 4.4.2
    dimensions(sizes): time_counter(24), y(266), x(256)
    variables(dimensions): float32 atmpres(time_counter,y,x), float64 nav_lat(y,x), float64 nav_lon(y,x), float32 percentcloud(time_counter,y,x), float32 precip(time_counter,y,x), float32 qair(time_counter,y,x), float32 solar(time_counter,y,x), float32 tair(time_counter,y,x), float32 therm_rad(time_counter,y,x), float64 time_counter(time_counter), float32 u_wind(time_counter,y,x), float32 v_wind(time_counter,y,x), float64 x(x), float64 y(y)
    groups: 

./TEPLOFILES/operational/ops_y2017m12d24.nc
358
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file fo